In [ ]:
"""
1. input files
2. load data in that line
3. save data


"""

# 1 xspec

In [36]:
import numpy as np
from glob import glob
import re
import pandas as pd

In [81]:
def tidy_dict2df(savepath, regname, mdl, bigkeys = ['T', 'Z', 'n']):
    # initialize dict
    output_dict = {'reg':[], 'cstat':[], 'dof':[]}
    for name in bigkeys:
        output_dict[f'{name}-errlo'] = []
        output_dict[f'{name}-errhi'] = []
        output_dict[f'{name}-value'] = []
        output_dict[f'{name}-value'] = []
    

    cts = 0
    dirs = glob(f'{savepath}/SDSS*')
    for dir in dirs:
        srcname = dir.split('/')[-1]
        val_file = f'{dir}/fit_231115/logs/annu_{regname}_{mdl}_freepar.log'
        err_file = f'{dir}/fit_231115/logs/annu_{regname}_{mdl}_chain1000_par.log'
        cst_file = f'{dir}/fit_231115/logs/annu_{regname}_{mdl}_fit.log'

        # read mcmc errors
        if glob(val_file) and glob(err_file):
            output_dict[f'reg'].append(f'{srcname}')
            # read the annuli from eckert regionfile
            with open(err_file) as f:
                lines = f.readlines()
            for i in range(len(bigkeys)):
                if len(lines)> 20:
                    errlo = 999
                    errhi = 999
                else:
                    start_idx = next((index for index, line in enumerate(lines) if 'Parameter' in line), None)
                    
                    errlo = lines[int(start_idx+1+i)].split('(')[-1].split(',')[0]
                    errhi = lines[int(start_idx+1+i)].split('(')[-1].split(',')[-1][:-2]
                output_dict[f'{bigkeys[i]}-errlo'].append(abs(float(errlo)))
                output_dict[f'{bigkeys[i]}-errhi'].append(float(errhi))

            # read value
            with open(val_file) as f:
                text = f.read()
            pattern = r'([+-]?[\d]*\.?[\d]+(?:[eE][-+]?\d+)?)\s+\+/-'
            values = re.findall(pattern, text)
            for i in range(len(bigkeys)):
                output_dict[f'{bigkeys[i]}-value'].append(float(values[i]))
            
            # read cstat
            with open(cst_file) as f:
                lines = f.readlines()[-30:]
            for line in lines:
                if 'Total fit statistic' in line:
                    cstat = float(line.split('tic')[1].split('with')[0].strip())
                    dof = int(line.split('with')[-1].split('d.o.f')[0].strip())
                    output_dict['cstat'].append(cstat)
                    output_dict['dof'].append(dof)
        else:
            cts+=1
            continue
    print(cts)
    return output_dict

1.1 GDEM

In [84]:
my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'

new_dict = tidy_dict2df(my_rootdir, 'R500-01', 'GDEM',  bigkeys = ['T', 'Tsig', 'Z', 'n'])
new_df = pd.DataFrame(new_dict)
new_df.to_csv(f'{my_rootdir}/../dats/R500-01_GDEM_xspec.csv')

3


1.2 1T

In [85]:
my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'

new_dict = tidy_dict2df(my_rootdir, 'R500-01', '1T',  bigkeys = ['T', 'Z', 'n'])
new_df = pd.DataFrame(new_dict)
new_df.to_csv(f'{my_rootdir}/../dats/R500-01_1T_xspec.csv')

3


# 2 spex

In [ ]:
def tidy_dict2df(savepath, regname, mdl, bigkeys = ['T', 'Z', 'n']):
    # initialize dict
    output_dict = {'reg':[]}
    for name in bigkeys:
        output_dict[f'{name}-errlo'] = []
        output_dict[f'{name}-errhi'] = []
        output_dict[f'{name}-value'] = []
        output_dict[f'{name}-value'] = []
    

    cts = 0
    dirs = glob(f'{savepath}/SDSS*')
    for dir in dirs:
        srcname = dir.split('/')[-1]
        val_file = f'{dir}/fit_spex_231115/logs/annu_{regname}_{mdl}_freepar.log'
        err_file = f'{dir}/fit_spex_231115/logs/annu_{regname}_{mdl}_chain1000_par.log'
        # read mcmc errors
        if glob(val_file) and glob(err_file):
            output_dict[f'reg'].append(f'{srcname}')
            # read the annuli from eckert regionfile
            with open(err_file) as f:
                lines = f.readlines()
            for i in range(len(bigkeys)):
                if len(lines)> 20:
                    errlo = 999
                    errhi = 999
                else:
                    start_idx = next((index for index, line in enumerate(lines) if 'Parameter' in line), None)
                    
                    errlo = lines[int(start_idx+1+i)].split('(')[-1].split(',')[0]
                    errhi = lines[int(start_idx+1+i)].split('(')[-1].split(',')[-1][:-2]
                
                output_dict[f'{bigkeys[i]}-errlo'].append(abs(float(errlo)))
                output_dict[f'{bigkeys[i]}-errhi'].append(float(errhi))

            # read value
            with open(val_file) as f:
                text = f.read()
            pattern = r'([+-]?[\d]*\.?[\d]+(?:[eE][-+]?\d+)?)\s+\+/-'
            values = re.findall(pattern, text)
            for i in range(len(bigkeys)):
                output_dict[f'{bigkeys[i]}-value'].append(float(values[i]))
            
        else:
            cts+=1
            # print(dir)
            continue
    print(cts)
    return output_dict

2.1 GDEM

2.2 1T